In [1]:
from database import init, Post, load_sqlite_as_df
import requests
from tqdm import tqdm
from data_loader import pull_posts
import peewee

init()

Deleting data/news.db...
New DB created!


In [2]:
i = 1
batch_size = 10
with tqdm() as progress_bar:
    while True:
        if i > 5:
            print("Early stopping.")
            break

        try:
            parsed_posts = pull_posts(i, batch_size=batch_size, order="asc")
            Post.bulk_create(
                list(map(lambda x: Post(**x), parsed_posts)), batch_size=batch_size
            )
        except peewee.IntegrityError:
            print("Duplicate entry")
        finally:
            i += 1
            progress_bar.update(batch_size)

50it [00:08,  5.68it/s]

Early stopping.


In [3]:
# convert sqlite to pandas

df = load_sqlite_as_df()
df.content

,id,link,date_gmt,title,summary,content,category,post_tags,syndication
0,1514,https://news.wisc.edu/dancers-and-musicians-to...,1997-02-01T06:00:00,Dancers and Musicians to Tribute Louise Kloepper,A special concert of music and dance will hono...,\n A special concert of music and dance will ...,['Campus news'],None,None
1,1515,https://news.wisc.edu/nobel-laureate-to-speak-...,1997-02-01T06:00:00,Nobel Laureate to Speak on ‘Chemistry as a Lib...,Nobel Prize-winning chemist Dudley Herschbach ...,\n Nobel Prize-winning chemist Dudley Herschb...,['Campus news'],None,None
2,1516,https://news.wisc.edu/spring-enrollment-slight...,1997-02-03T06:00:00,Spring Enrollment Slightly Up,Preliminary spring semester enrollment figures...,\n Preliminary spring semester enrollment fig...,['Campus news'],None,None
3,1517,https://news.wisc.edu/hong-kong-film-director-...,1997-02-03T06:00:00,Hong Kong Film Director Ann Hui to Visit UW-Ma...,Asia’s most celebrated female film director wi...,\n Asia’s most celebrated female film directo...,['Campus news'],None,None
4,1518,https://news.wisc.edu/uw-pro-arte-to-perform-b...,1997-02-03T06:00:00,UW Pro Arte to Perform Beethoven String Quartets,UW-Madison’s renowned Pro Arte will present th...,"\n Russia, 1812. Acclaimed cellist Bernhard R...",['Campus news'],None,None
5,1519,https://news.wisc.edu/cell-transplant-therapy-...,1997-02-03T06:00:00,Cell Transplant Therapy May Have Applications ...,In a hopeful new development for people with m...,\n In a hopeful new development for people wi...,['Campus news'],None,None
6,1520,https://news.wisc.edu/uw-madison-economist-app...,1997-02-06T06:00:00,UW-Madison Economist Appointed to the U.S. Tre...,Economist John Karl Scholz has been named by t...,\n Economist John Karl Scholz has been named ...,['Campus news'],None,None
7,1521,https://news.wisc.edu/henry-barschall-pioneeri...,1997-02-06T06:00:00,"Henry Barschall, Pioneering Nuclear Physicist,...","Professor Henry H. Barschall, one of the world...","\n\n\n\n Professor Henry H. Barschall, one of...",['Campus news'],None,None
8,1522,https://news.wisc.edu/fitzpatrick-named-letter...,1997-02-06T06:00:00,Fitzpatrick Named Letters and Science Associat...,"Mary Anne Fitzpatrick, an expert in family com...","\nMary Anne Fitzpatrick, an expert in family c...",['Campus news'],None,None
9,1523,https://news.wisc.edu/uw-madison-rated-third-b...,1997-02-07T06:00:00,UW-Madison Rated Third Best Value in Country,"Weighing both cost and academic quality, the M...",,['Campus news'],None,None


In [4]:
df.content[0]

'\n  A special concert of music and dance will honor the memory of Louise Kloepper, former chair of the University of Wisconsin- Madison Dance Program, on Jan. 19, what would have been her 87th birthday.\n\n\n  Kloepper died Dec. 15. She came to the UW in 1942 to study dance, after almost a decade teaching and dancing with the Hanya Holm School and Company.\n\n\n  According to Mary Alice Brennan, professor of dance, “Kloepper brought to the Dance Program the talent, experience and insight of a professional dancer. First as a student, then a professor and later as chair of the program, she was the next generation following in the footsteps of Margaret H’Doubler,” who founded UW’s Dance Program in 1926.\n\n\n  Kloepper earned her bachelor of science degree in dance from Wisconsin in 1946, and the same year became an assistant professor in the program. Until her retirement in 1975, Kloepper was a distinguished teacher of dance technique and composition. She served as artistic director and